# ライブラリのインポート / データの読み込み

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
import numpy as np
import pandas as pd

pd.set_option('max_colwidth', 500)
pd.set_option('max_columns', 500)
pd.set_option('max_rows', 500)

%matplotlib inline

from matplotlib import pyplot as plt
import matplotlib.ticker as mtick # For specifying the axes tick format 

import seaborn as sns
import re

import json, os, gc, math, time
import datetime
import collections
from tqdm import tqdm
import glob

from statistics import mean
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import KFold, GroupKFold, StratifiedKFold

from sklearn import metrics
import time

import lightgbm as lgb

import warnings
warnings.filterwarnings("ignore")

In [3]:
#ドライブへのアクセス
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/00_datascience/19_ufj_bank"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/00_datascience/19_ufj_bank


In [4]:
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')
sub = pd.read_csv('input/sample_submit.csv')

# bertによる特徴抽出

In [5]:
def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

In [6]:
for i in range(len(train)):
  train.loc[i, 'cleaned_text'] = remove_html(train.loc[i, 'html_content'])

for i in range(len(test)):
  test.loc[i, 'cleaned_text'] = remove_html(test.loc[i, 'html_content'])

In [7]:
#すでに抽出済みの特徴量を使用
text_train_df = pd.read_csv('input/05_seq_train_df.csv')
text_train_df.drop(['Unnamed: 0'], axis=1, inplace=True)
text_test_df = pd.read_csv('input/05_seq_test_df.csv')
text_test_df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [8]:
merge_text = pd.concat([text_train_df, text_test_df], ignore_index=True)

# ラベルデータの前処理

In [9]:
#学習しやすいようにtrain, testをマージ
merge_df = pd.concat([train, test], ignore_index=True)

In [10]:
#goalカラムの数値化
for i in range(len(merge_df)):
  price_list = re.findall(r"\d+", merge_df.loc[i, 'goal'])
  merge_df.loc[i, 'goal_min'] = price_list[0]
  if len(price_list) == 1:
    merge_df.loc[i, 'goal_max'] = price_list[0]
  else:
    merge_df.loc[i, 'goal_max'] = price_list[1]

In [11]:
#int型に変換
merge_df['goal_max'] = merge_df['goal_max'].astype(int)
merge_df['goal_min'] = merge_df['goal_min'].astype(int)

#1日あたりの目標金額の算出
merge_df['goal_per_day_max'] = merge_df['goal_max'] / merge_df['duration']
merge_df['goal_per_day_min'] = merge_df['goal_min'] / merge_df['duration']
#merge_df.head()

In [12]:
# ワンホットエンコーディング

dummy_cols = ['goal',	'country', 'category1',	'category2']
from sklearn.preprocessing import OneHotEncoder

dummy_df = pd.get_dummies(merge_df[dummy_cols])

In [13]:
merge_df_new = pd.merge(merge_df, dummy_df, left_index=True, right_index=True)

In [14]:
#ラベルエンコーディング
from sklearn.preprocessing import LabelEncoder

le_cols = ['goal', 'country', 'category1', 'category2']

#カテゴリ変数をループしてlabel encoding
for c in le_cols:
  le = LabelEncoder()
  le.fit(merge_df_new[c])
  merge_df_new[c] = le.transform(merge_df_new[c])

# tfidfによる特徴抽出

In [15]:
#tfidfによる特徴抽出
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer()
features = tv.fit_transform(merge_df_new["cleaned_text"].fillna(""))

In [16]:
from sklearn.decomposition import TruncatedSVD, NMF, LatentDirichletAllocation
from sklearn.pipeline import Pipeline



tfidf_svd = Pipeline(steps=[
    ("TfidfVectorizer", TfidfVectorizer()),
    ("TruncatedSVD", TruncatedSVD(n_components=1000, random_state=42))
])

features_svd = tfidf_svd.fit_transform(merge_df_new["cleaned_text"].fillna(""))

In [17]:
svd_df_merge = pd.DataFrame(features_svd)
svd_df_merge.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
0,0.294975,0.000612,0.023093,-0.043301,-0.004650,0.020541,-0.045337,0.023749,-0.013548,0.034168,-0.013013,0.006800,-0.034913,0.006792,-0.015054,0.005607,-0.006633,0.028449,0.020395,-0.021418,0.013802,0.013191,-0.021499,0.007818,-0.033270,0.021218,-0.028720,0.018364,0.040835,0.015443,-0.009399,-0.024765,-0.008274,0.021418,-0.004099,-0.001800,0.006569,-0.017899,-0.022700,-0.039155,0.027662,-0.042093,-0.018316,0.007389,-0.015836,0.025763,0.019476,-0.021724,-0.004165,-0.040945,-0.011895,-0.005414,0.044285,0.000752,-0.025620,-0.012126,-0.016218,-0.032687,0.000541,0.013212,0.023160,0.008573,0.019850,0.001818,0.028804,0.006221,-0.017187,-0.031764,-0.003156,-0.015492,-0.026320,0.001645,-0.009675,-0.020839,0.005702,-0.031384,-0.014141,0.006166,-0.016562,0.021373,-0.019713,-0.013808,-0.013545,0.004556,0.024441,0.002668,0.007806,0.010765,-0.034016,-0.008416,-0.018521,-0.010729,-0.014250,-0.011515,0.015059,0.008557,0.014118,0.006811,-0.011104,-0.010446,-0.004324,-0.017287,-0.010474,-0.004138,-0.010734,-0.004927,0.010384,-0.017388,-0.008818,0.014501,0.016185,-0.012236,-0.001071,-0.003736,-0.004962,-0.001863,0.005490,-0.005913,0.011078,-0.000654,0.015868,-0.015941,-0.010237,-0.028429,0.002320,-0.006602,-0.037504,-0.015901,-0.001221,0.005836,-0.002604,0.033665,0.016386,0.011442,0.009914,-0.007815,-0.002609,-0.007079,0.006088,-0.030331,-0.013934,-0.017419,0.000299,-0.021974,0.014502,-0.016659,0.015314,-0.012596,-0.014667,-0.034675,-0.010979,-0.020586,-0.019136,-0.012399,-0.005613,0.002580,0.005750,-0.008406,-0.008458,-0.013055,-0.027402,-0.003059,0.002124,0.010547,-0.012725,-0.003126,0.016835,0.004937,-0.005492,-0.016231,0.017594,-0.005843,0.005354,-0.001336,0.014849,-0.014394,0.005559,-0.000798,0.019274,-0.007828,-0.013374,0.000941,-0.006692,-0.015561,-0.009937,0.028827,-0.003553,-0.008625,0.015199,-0.004971,0.011670,0.001208,0.014611,-0.005897,-0.014305,-0.000319,-0.002148,0.023332,0.001524,-0.009723,-0.013181,0.004239,0.002568,0.009010,-0.015208,-0.002074,0.031430,-0.028039,0.012889,-0.000447,0.034450,-0.047279,0.012189,-0.000357,-0.016639,-0.040275,-0.000082,-0.018842,-0.002570,0.

# w2vによる特徴抽出

https://qiita.com/propella/items/febc423998fd210800ca
よりフォーク

In [26]:
#単語に対して分散表現を与える
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(merge_df_new["cleaned_text"])]
model = Doc2Vec(documents, vector_size=100, window=5, min_count=1, workers=4)

In [ ]:
#コサイン類似度が上位n個のものを抽出
n = 10
for i in range(len(merge_df_new)):
  print(i)
  for j in range(n):
    merge_df_new.loc[i, 'similar{}'.format(j)] = model.docvecs.most_similar(i)[j][0]

In [28]:
merge_df_new.head()

,id,goal,country,duration,category1,category2,html_content,state,cleaned_text,goal_min,goal_max,goal_per_day_max,goal_per_day_min,goal_1-1000,goal_100000+,goal_10001-11000,goal_1001-2000,goal_11001-12000,goal_12001-13000,goal_13001-14000,goal_14001-15000,goal_15001-16000,goal_16001-17000,goal_17001-18000,goal_18001-19000,goal_19001-20000,goal_20001-21000,goal_2001-3000,goal_21001-22000,goal_22001-23000,goal_23001-24000,goal_24001-25000,goal_25001-26000,goal_26001-27000,goal_27001-28000,goal_28001-29000,goal_29001-30000,goal_30001-31000,goal_3001-4000,goal_31001-32000,goal_32001-33000,goal_33001-34000,goal_34001-35000,goal_35001-36000,goal_36001-37000,goal_37001-38000,goal_38001-39000,goal_39001-40000,goal_40001-41000,goal_4001-5000,goal_41001-42000,goal_42001-43000,goal_43001-44000,goal_44001-45000,goal_45001-46000,goal_46001-47000,goal_47001-48000,goal_48001-49000,goal_49001-50000,goal_50001-51000,goal_5001-6000,goal_51001-52000,goal_52001-53000,goal_53001-54000,goal_54001-55000,goal_55001-56000,goal_56001-57000,goal_57001-58000,goal_58001-59000,goal_59001-60000,goal_60001-61000,goal_6001-7000,goal_61001-62000,goal_62001-63000,goal_63001-64000,goal_64001-65000,goal_65001-66000,goal_66001-67000,goal_67001-68000,goal_68001-69000,goal_69001-70000,goal_70001-71000,goal_7001-8000,goal_71001-72000,goal_72001-73000,goal_73001-74000,goal_74001-75000,goal_75001-76000,goal_76001-77000,goal_77001-78000,goal_78001-79000,goal_79001-80000,goal_80001-81000,goal_8001-9000,goal_81001-82000,goal_82001-83000,goal_83001-84000,goal_84001-85000,goal_85001-86000,goal_86001-87000,goal_87001-88000,goal_88001-89000,goal_89001-90000,goal_90001-91000,goal_9001-10000,goal_91001-92000,goal_92001-93000,goal_93001-94000,goal_94001-95000,goal_95001-96000,goal_96001-97000,goal_97001-98000,goal_98001-99000,goal_99001-100000,country_AT,country_AU,country_BE,country_CA,country_CH,country_DE,country_DK,country_ES,country_FR,country_GB,country_HK,country_IE,country_IT,country_JP,country_LU,country_MX,country_NL,country_NO,country_NZ,country_SE,country_SG,country_US,category1_art,category1_comics,category1_crafts,category1_dance,category1_design,category1_fashion,category1_film & video,category1_food,category1_games,category1_journalism,category1_music,category1_photography,category1_publishing,category1_technology,category1_theater,category2_3d printing,category2_academic,category2_accessories,category2_action,category2_animals,category2_animation,category2_anthologies,category2_apparel,category2_apps,category2_architecture,category2_art books,category2_audio,category2_bacon,category2_blues,category2_calendars,category2_camera equipment,category2_candles,category2_ceramics,category2_children's books,category2_childrenswear,category2_chiptune,category2_civic design,category2_classical music,category2_comedy,category2_comic books,category2_community gardens,category2_conceptual art,category2_cookbooks,category2_country & folk,category2_couture,category2_crochet,category2_digital art,category2_diy,category2_diy electronics,category2_documentary,category2_drama,category2_drinks,category2_electronic music,category2_embroidery,category2_events,category2_experimental,category2_fabrication tools,category2_faith,category2_family,category2_fantasy,category2_farmer's markets,category2_farms,category2_festivals,category2_fiction,category2_fine art,category2_flight,category2_food trucks,category2_footwear,category2_gadgets,category2_gaming hardware,category2_glass,category2_graphic design,category2_graphic novels,category2_hardware,category2_hip-hop,category2_horror,category2_illustration,category2_immersive,category2_indie rock,category2_installations,category2_interactive design,category2_jazz,category2_jewelry,category2_kids,category2_knitting,category2_latin,category2_letterpress,category2_literary journals,category2_literary spaces,category2_live games,category2_makerspaces,category2_metal,category2_mixed media,category2_mobile games,category2_movie theaters,category2_mus

# データのマージ/kmeans用の前処理

In [29]:
#不要なカラムの定義
non_use_cols = ['id',	'html_content',	'cleaned_text', 'state']

#スケーリングするカラムの選択
sc_cols = ['duration', 'goal_max', 'goal_min', 'goal_per_day_max', 'goal_per_day_min']

In [30]:
#bertから抽出した特徴量を追加
full_merge_df = pd.merge(merge_df_new.drop(non_use_cols, axis=1), merge_text, left_index=True, right_index=True)

In [31]:
#tfidfから抽出した特徴量を追加
full_merge_df_fin = pd.merge(full_merge_df, svd_df_merge, left_index=True, right_index=True)

In [32]:
full_merge_df_fin.head()

,goal,country,duration,category1,category2,goal_min,goal_max,goal_per_day_max,goal_per_day_min,goal_1-1000,goal_100000+,goal_10001-11000,goal_1001-2000,goal_11001-12000,goal_12001-13000,goal_13001-14000,goal_14001-15000,goal_15001-16000,goal_16001-17000,goal_17001-18000,goal_18001-19000,goal_19001-20000,goal_20001-21000,goal_2001-3000,goal_21001-22000,goal_22001-23000,goal_23001-24000,goal_24001-25000,goal_25001-26000,goal_26001-27000,goal_27001-28000,goal_28001-29000,goal_29001-30000,goal_30001-31000,goal_3001-4000,goal_31001-32000,goal_32001-33000,goal_33001-34000,goal_34001-35000,goal_35001-36000,goal_36001-37000,goal_37001-38000,goal_38001-39000,goal_39001-40000,goal_40001-41000,goal_4001-5000,goal_41001-42000,goal_42001-43000,goal_43001-44000,goal_44001-45000,goal_45001-46000,goal_46001-47000,goal_47001-48000,goal_48001-49000,goal_49001-50000,goal_50001-51000,goal_5001-6000,goal_51001-52000,goal_52001-53000,goal_53001-54000,goal_54001-55000,goal_55001-56000,goal_56001-57000,goal_57001-58000,goal_58001-59000,goal_59001-60000,goal_60001-61000,goal_6001-7000,goal_61001-62000,goal_62001-63000,goal_63001-64000,goal_64001-65000,goal_65001-66000,goal_66001-67000,goal_67001-68000,goal_68001-69000,goal_69001-70000,goal_70001-71000,goal_7001-8000,goal_71001-72000,goal_72001-73000,goal_73001-74000,goal_74001-75000,goal_75001-76000,goal_76001-77000,goal_77001-78000,goal_78001-79000,goal_79001-80000,goal_80001-81000,goal_8001-9000,goal_81001-82000,goal_82001-83000,goal_83001-84000,goal_84001-85000,goal_85001-86000,goal_86001-87000,goal_87001-88000,goal_88001-89000,goal_89001-90000,goal_90001-91000,goal_9001-10000,goal_91001-92000,goal_92001-93000,goal_93001-94000,goal_94001-95000,goal_95001-96000,goal_96001-97000,goal_97001-98000,goal_98001-99000,goal_99001-100000,country_AT,country_AU,country_BE,country_CA,country_CH,country_DE,country_DK,country_ES,country_FR,country_GB,country_HK,country_IE,country_IT,country_JP,country_LU,country_MX,country_NL,country_NO,country_NZ,country_SE,country_SG,country_US,category1_art,category1_comics,category1_crafts,category1_dance,category1_design,category1_fashion,category1_film & video,category1_food,category1_games,category1_journalism,category1_music,category1_photography,category1_publishing,category1_technology,category1_theater,category2_3d printing,category2_academic,category2_accessories,category2_action,category2_animals,category2_animation,category2_anthologies,category2_apparel,category2_apps,category2_architecture,category2_art books,category2_audio,category2_bacon,category2_blues,category2_calendars,category2_camera equipment,category2_candles,category2_ceramics,category2_children's books,category2_childrenswear,category2_chiptune,category2_civic design,category2_classical music,category2_comedy,category2_comic books,category2_community gardens,category2_conceptual art,category2_cookbooks,category2_country & folk,category2_couture,category2_crochet,category2_digital art,category2_diy,category2_diy electronics,category2_documentary,category2_drama,category2_drinks,category2_electronic music,category2_embroidery,category2_events,category2_experimental,category2_fabrication tools,category2_faith,category2_family,category2_fantasy,category2_farmer's markets,category2_farms,category2_festivals,category2_fiction,category2_fine art,category2_flight,category2_food trucks,category2_footwear,category2_gadgets,category2_gaming hardware,category2_glass,category2_graphic design,category2_graphic novels,category2_hardware,category2_hip-hop,category2_horror,category2_illustration,category2_immersive,category2_indie rock,category2_installations,category2_interactive design,category2_jazz,category2_jewelry,category2_kids,category2_knitting,category2_latin,category2_letterpress,category2_literary journals,category2_literary spaces,category2_live games,category2_makerspaces,category2_metal,category2_mixed media,category2_mobile games,category2_movie theaters,category2_music videos,category2_musical,categor

In [33]:
#sc_colsのスケーリング
from sklearn.preprocessing import StandardScaler

for c in sc_cols:
  std = StandardScaler()
  full_merge_df_fin[c] = std.fit_transform(pd.DataFrame(full_merge_df_fin[c]))

# kmeansによるクラスタリング

In [40]:
from sklearn.cluster import KMeans
non_use_cols_for_km = [
    'goal', 
    'country', 
    'category1', 
    'category2', 
    'similar0', 'similar1', 'similar2', 'similar3', 'similar4', 'similar5', 'similar6', 'similar7', 'similar8', 'similar9']

In [41]:
# エルボー方による推定。クラスター数を1から20に増やして、それぞれの距離の総和を求める
"""
dist_list =[]
for i in range(1,50):
    print('今はクラスター数{}のクラスタリングを実行しています。'.format(i))
    kmeans= KMeans(n_clusters=i, init='random', random_state=0)
    kmeans.fit(full_merge_df_fin.drop(non_use_cols_for_km, axis=1))
    dist_list.append(kmeans.inertia_)
    
# グラフを表示
plt.plot(range(1,50), dist_list,marker='+')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')
"""

"\ndist_list =[]\nfor i in range(1,50):\n    print('今はクラスター数{}のクラスタリングを実行しています。'.format(i))\n    kmeans= KMeans(n_clusters=i, init='random', random_state=0)\n    kmeans.fit(full_merge_df_fin.drop(non_use_cols_for_km, axis=1))\n    dist_list.append(kmeans.inertia_)\n    \n# グラフを表示\nplt.plot(range(1,50), dist_list,marker='+')\nplt.xlabel('Number of clusters')\nplt.ylabel('Distortion')\n"

In [43]:
#クラスタリング
clusters_list=[5, 10, 15, 20, 30, 50, 100]

for n_clusters in clusters_list:
  print('今クラスター数[{}]のクラスタリングをしています。'.format(n_clusters))
  kmeans = KMeans(init='random', n_clusters=n_clusters, random_state=0)
  kmeans.fit(full_merge_df_fin.drop(non_use_cols_for_km, axis=1))
  full_merge_df_fin['cluster_number_{}'.format(n_clusters)] =pd.Series(kmeans.labels_, name='cluster_number_{}'.format(5))

今クラスター数[5]のクラスタリングをしています。
今クラスター数[10]のクラスタリングをしています。
今クラスター数[15]のクラスタリングをしています。
今クラスター数[20]のクラスタリングをしています。
今クラスター数[30]のクラスタリングをしています。
今クラスター数[50]のクラスタリングをしています。
今クラスター数[100]のクラスタリングをしています。


# aggrigation特徴量の作成

In [44]:
#aggrigationの際にgroup化するカラム
group_cols = ['goal',	'country', 'category1',	'category2', 'cluster_number_5', 'cluster_number_10', 'cluster_number_15', 'cluster_number_20', 'cluster_number_30', 'cluster_number_50', 'cluster_number_100', 'similar0', 'similar1', 'similar2', 'similar3', 'similar4', 'similar5', 'similar6', 'similar7', 'similar8', 'similar9']

In [45]:
#関数定義
def agg_func_duration(input_df, cols):
  agg_func = ["mean", "median", "max", "min", "std"]
  tmp = input_df.groupby([c])["duration"].agg(agg_func)
  output_df = pd.merge(input_df, tmp, how="left", on=[c])[agg_func].add_suffix("_agg_func_duration")

  return output_df

def agg_func_state(input_df, cols):
  agg_func = ["mean", "median", "max", "min", "std"]
  tmp = input_df.groupby([c])["duration"].agg(agg_func)
  output_df = pd.merge(input_df, tmp, how="left", on=[c])[agg_func].add_suffix("_agg_func_state")

  return output_df

def agg_func_goal_per_day_max(input_df, cols):
  agg_func = ["mean", "median", "max", "min", "std"]
  tmp = input_df.groupby([c])["goal_per_day_max"].agg(agg_func)
  output_df = pd.merge(input_df, tmp, how="left", on=[c])[agg_func].add_suffix("_agg_func_state")

  return output_df

def agg_func_goal_per_day_min(input_df, cols):
  agg_func = ["mean", "median", "max", "min", "std"]
  tmp = input_df.groupby([c])["goal_per_day_min"].agg(agg_func)
  output_df = pd.merge(input_df, tmp, how="left", on=[c])[agg_func].add_suffix("_agg_func_state")

  return output_df

In [46]:
#実行の準備
full_merge_df_fin["state"] = train['state']
agg_df = agg_func_duration(full_merge_df_fin, group_cols[0])

In [47]:
#関数の実行
for c in group_cols[1:len(group_cols)]:
  agg_df = pd.merge(agg_df, agg_func_duration(full_merge_df_fin, c), left_index=True, right_index=True)

for c in group_cols:
  agg_df = pd.merge(agg_df, agg_func_state(full_merge_df_fin, c), left_index=True, right_index=True)

for c in group_cols:
  agg_df = pd.merge(agg_df, agg_func_goal_per_day_max(full_merge_df_fin, c), left_index=True, right_index=True)

for c in group_cols:
  agg_df = pd.merge(agg_df, agg_func_goal_per_day_min(full_merge_df_fin, c), left_index=True, right_index=True)

In [48]:
agg_df

,mean_agg_func_duration_x,median_agg_func_duration_x,max_agg_func_duration_x,min_agg_func_duration_x,std_agg_func_duration_x,mean_agg_func_duration_y,median_agg_func_duration_y,max_agg_func_duration_y,min_agg_func_duration_y,std_agg_func_duration_y,mean_agg_func_duration_x,median_agg_func_duration_x,max_agg_func_duration_x,min_agg_func_duration_x,std_agg_func_duration_x,mean_agg_func_duration_y,median_agg_func_duration_y,max_agg_func_duration_y,min_agg_func_duration_y,std_agg_func_duration_y,mean_agg_func_duration_x,median_agg_func_duration_x,max_agg_func_duration_x,min_agg_func_duration_x,std_agg_func_duration_x,mean_agg_func_duration_y,median_agg_func_duration_y,max_agg_func_duration_y,min_agg_func_duration_y,std_agg_func_duration_y,mean_agg_func_duration_x,median_agg_func_duration_x,max_agg_func_duration_x,min_agg_func_duration_x,std_agg_func_duration_x,mean_agg_func_duration_y,median_agg_func_duration_y,max_agg_func_duration_y,min_agg_func_duration_y,std_agg_func_duration_y,mean_agg_func_duration_x,median_agg_func_duration_x,max_agg_func_duration_x,min_agg_func_duration_x,std_agg_func_duration_x,mean_agg_func_duration_y,median_agg_func_duration_y,max_agg_func_duration_y,min_agg_func_duration_y,std_agg_func_duration_y,mean_agg_func_duration_x,median_agg_func_duration_x,max_agg_func_duration_x,min_agg_func_duration_x,std_agg_func_duration_x,mean_agg_func_duration_y,median_agg_func_duration_y,max_agg_func_duration_y,min_agg_func_duration_y,std_agg_func_duration_y,mean_agg_func_duration_x,median_agg_func_duration_x,max_agg_func_duration_x,min_agg_func_duration_x,std_agg_func_duration_x,mean_agg_func_duration_y,median_agg_func_duration_y,max_agg_func_duration_y,min_agg_func_duration_y,std_agg_func_duration_y,mean_agg_func_duration_x,median_agg_func_duration_x,max_agg_func_duration_x,min_agg_func_duration_x,std_agg_func_duration_x,mean_agg_func_duration_y,median_agg_func_duration_y,max_agg_func_duration_y,min_agg_func_duration_y,std_agg_func_duration_y,mean_agg_func_duration_x,median_agg_func_duration_x,max_agg_func_duration_x,min_agg_func_duration_x,std_agg_func_duration_x,mean_agg_func_duration_y,median_agg_func_duration_y,max_agg_func_duration_y,min_agg_func_duration_y,std_agg_func_duration_y,mean_agg_func_duration_x,median_agg_func_duration_x,max_agg_func_duration_x,min_agg_func_duration_x,std_agg_func_duration_x,mean_agg_func_duration_y,median_agg_func_duration_y,max_agg_func_duration_y,min_agg_func_duration_y,std_agg_func_duration_y,mean_agg_func_duration,median_agg_func_duration,max_agg_func_duration,min_agg_func_duration,std_agg_func_duration,mean_agg_func_state_x,median_agg_func_state_x,max_agg_func_state_x,min_agg_func_state_x,std_agg_func_state_x,mean_agg_func_state_y,median_agg_func_state_y,max_agg_func_state_y,min_agg_func_state_y,std_agg_func_state_y,mean_agg_func_state_x,median_agg_func_state_x,max_agg_func_state_x,min_agg_func_state_x,std_agg_func_state_x,mean_agg_func_state_y,median_agg_func_state_y,max_agg_func_state_y,min_agg_func_state_y,std_agg_func_state_y,mean_agg_func_state_x,median_agg_func_state_x,max_agg_func_state_x,min_agg_func_state_x,std_agg_func_state_x,mean_agg_func_state_y,median_agg_func_state_y,max_agg_func_state_y,min_agg_func_state_y,std_agg_func_state_y,mean_agg_func_state_x,median_agg_func_state_x,max_agg_func_state_x,min_agg_func_state_x,std_agg_func_state_x,mean_agg_func_state_y,median_agg_func_state_y,max_agg_func_state_y,min_agg_func_state_y,std_agg_func_state_y,mean_agg_func_state_x,median_agg_func_state_x,max_agg_func_state_x,min_agg_func_state_x,std_agg_func_state_x,mean_agg_func_state_y,median_agg_func_state_y,max_agg_func_state_y,min_agg_func_state_y,std_agg_func_state_y,mean_agg_func_state_x,median_agg_func_state_x,max_agg_func_state_x,min_agg_func_state_x,std_agg_func_state_x,mean_agg_func_state_y,median_agg_func_state_y,max_agg_func_state_y,min_agg_func_state_y,std_agg_func_state_y,mean_agg_func_state_x,median_agg_func_state_x,max_agg_func_state_x,min_agg_func_state_x,std_agg_f

In [49]:
#データのマージ
full_merge_df_fin = pd.merge(full_merge_df_fin, agg_df, left_index=True, right_index=True)
full_merge_df_fin.drop(['state'], axis=1, inplace=True)

# データセットの整理

In [50]:
train_df = pd.DataFrame(full_merge_df_fin.loc[:len(train),])
test_df = pd.DataFrame(full_merge_df_fin.loc[len(train):19591,])

In [51]:
test_df = test_df.reset_index()
test_df.drop(['index'], axis=1, inplace=True)
test_df.head()

,goal,country,duration,category1,category2,goal_min,goal_max,goal_per_day_max,goal_per_day_min,goal_1-1000,goal_100000+,goal_10001-11000,goal_1001-2000,goal_11001-12000,goal_12001-13000,goal_13001-14000,goal_14001-15000,goal_15001-16000,goal_16001-17000,goal_17001-18000,goal_18001-19000,goal_19001-20000,goal_20001-21000,goal_2001-3000,goal_21001-22000,goal_22001-23000,goal_23001-24000,goal_24001-25000,goal_25001-26000,goal_26001-27000,goal_27001-28000,goal_28001-29000,goal_29001-30000,goal_30001-31000,goal_3001-4000,goal_31001-32000,goal_32001-33000,goal_33001-34000,goal_34001-35000,goal_35001-36000,goal_36001-37000,goal_37001-38000,goal_38001-39000,goal_39001-40000,goal_40001-41000,goal_4001-5000,goal_41001-42000,goal_42001-43000,goal_43001-44000,goal_44001-45000,goal_45001-46000,goal_46001-47000,goal_47001-48000,goal_48001-49000,goal_49001-50000,goal_50001-51000,goal_5001-6000,goal_51001-52000,goal_52001-53000,goal_53001-54000,goal_54001-55000,goal_55001-56000,goal_56001-57000,goal_57001-58000,goal_58001-59000,goal_59001-60000,goal_60001-61000,goal_6001-7000,goal_61001-62000,goal_62001-63000,goal_63001-64000,goal_64001-65000,goal_65001-66000,goal_66001-67000,goal_67001-68000,goal_68001-69000,goal_69001-70000,goal_70001-71000,goal_7001-8000,goal_71001-72000,goal_72001-73000,goal_73001-74000,goal_74001-75000,goal_75001-76000,goal_76001-77000,goal_77001-78000,goal_78001-79000,goal_79001-80000,goal_80001-81000,goal_8001-9000,goal_81001-82000,goal_82001-83000,goal_83001-84000,goal_84001-85000,goal_85001-86000,goal_86001-87000,goal_87001-88000,goal_88001-89000,goal_89001-90000,goal_90001-91000,goal_9001-10000,goal_91001-92000,goal_92001-93000,goal_93001-94000,goal_94001-95000,goal_95001-96000,goal_96001-97000,goal_97001-98000,goal_98001-99000,goal_99001-100000,country_AT,country_AU,country_BE,country_CA,country_CH,country_DE,country_DK,country_ES,country_FR,country_GB,country_HK,country_IE,country_IT,country_JP,country_LU,country_MX,country_NL,country_NO,country_NZ,country_SE,country_SG,country_US,category1_art,category1_comics,category1_crafts,category1_dance,category1_design,category1_fashion,category1_film & video,category1_food,category1_games,category1_journalism,category1_music,category1_photography,category1_publishing,category1_technology,category1_theater,category2_3d printing,category2_academic,category2_accessories,category2_action,category2_animals,category2_animation,category2_anthologies,category2_apparel,category2_apps,category2_architecture,category2_art books,category2_audio,category2_bacon,category2_blues,category2_calendars,category2_camera equipment,category2_candles,category2_ceramics,category2_children's books,category2_childrenswear,category2_chiptune,category2_civic design,category2_classical music,category2_comedy,category2_comic books,category2_community gardens,category2_conceptual art,category2_cookbooks,category2_country & folk,category2_couture,category2_crochet,category2_digital art,category2_diy,category2_diy electronics,category2_documentary,category2_drama,category2_drinks,category2_electronic music,category2_embroidery,category2_events,category2_experimental,category2_fabrication tools,category2_faith,category2_family,category2_fantasy,category2_farmer's markets,category2_farms,category2_festivals,category2_fiction,category2_fine art,category2_flight,category2_food trucks,category2_footwear,category2_gadgets,category2_gaming hardware,category2_glass,category2_graphic design,category2_graphic novels,category2_hardware,category2_hip-hop,category2_horror,category2_illustration,category2_immersive,category2_indie rock,category2_installations,category2_interactive design,category2_jazz,category2_jewelry,category2_kids,category2_knitting,category2_latin,category2_letterpress,category2_literary journals,category2_literary spaces,category2_live games,category2_makerspaces,category2_metal,category2_mixed media,category2_mobile games,category2_movie theaters,category2_music videos,category2_musical,categor

# lgbによる予測

In [52]:
target = train['state']

In [53]:
# 3分割交差検証を指定し、インスタンス化
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

kf = KFold(n_splits=5)  # 3分割交差検証のためにインスタンス化

# スコアとモデルを格納するリスト
score_list = []
models = []

print('input size is {}行, {}カラム'.format(len(train_df), len(train_df.columns)))
for fold_, (train_index, valid_index) in enumerate(kf.split(train, target)):
    train_x = train_df.iloc[train_index]
    valid_x = train_df.iloc[valid_index]
    train_y = target[train_index]
    valid_y = target[valid_index]
    
    print(f'fold{fold_ + 1} start')

    gbm = lgb.LGBMClassifier(objective='binary',num_boost_round=50000, learning_rate=0.1)
    gbm.fit(train_x, train_y, eval_set = [(valid_x, valid_y)],
                early_stopping_rounds=100,
                verbose= 100) # 学習の状況を表示しない
    
    oof = gbm.predict(valid_x, num_iteration=gbm.best_iteration_)
    score_list.append(round(accuracy_score(valid_y, oof)*100,2))
    models.append(gbm)  # 学習が終わったモデルをリストに入れておく
    print(f'fold{fold_ + 1} end\n' )
print(score_list, '平均score', np.mean(score_list), "%")  

input size is 9792行, 2497カラム
fold1 start
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.431735
Early stopping, best iteration is:
[69]	valid_0's binary_logloss: 0.430195
fold1 end

fold2 start
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.422785
[200]	valid_0's binary_logloss: 0.431981
Early stopping, best iteration is:
[117]	valid_0's binary_logloss: 0.419698
fold2 end

fold3 start
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.423383
[200]	valid_0's binary_logloss: 0.432444
Early stopping, best iteration is:
[131]	valid_0's binary_logloss: 0.420504
fold3 end

fold4 start
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.402257
[200]	valid_0's binary_logloss: 0.401789
Early stopping, best iteration is:
[125]	valid_0's binary_logloss: 0.396278
fold4 end

fold5 start
Training until validation

In [54]:
# testの予測
test_pred = np.zeros((len(test), 5))  # 行:len(test), 列:3のall zeroの配列を用意

for fold_, gbm in enumerate(models):  # 学習ずみのmodelをgbmに入れる
    pred_ = gbm.predict(test_df, num_iteration=gbm.best_iteration_)  # testの予測
    test_pred[:, fold_] = pred_  # １回目は0列目、2回目は1列目、2回目は3列目に格納

pred = (np.mean(test_pred, axis=1) > 0.5).astype(int)  # 平均をとって、0と１に変換

# submission

In [55]:
sub = pd.DataFrame(test['id'])

In [56]:
sub['state'] = pred
sub.head()

,id,state
0,test_00000,1
1,test_00001,1
2,test_00002,1
3,test_00003,0
4,test_00004,0


In [57]:
sub.to_csv('sub/14_lgb6.csv',index=False, header=None)